## Prepare the workspace

In [2]:
# Deep learning framework:
# torch: The core library for PyTorch, a popular deep learning framework. It provides
# tensors (similar to NumPy arrays but optimized for GPUs), as well as tools for building
# and training neural networks.
# Check torch version and CUDA status if GPU is enabled.
import torch
print(torch.__version__)
print(torch.cuda.is_available()) # Should return True when GPU is enabled. 

2.4.1+cpu
False


# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [3]:
# Package imports:
# Data handling and preprocessing:
# numpy: A powerful library for numerical computations. It provides support for arrays, matrices, 
# and a large collection of mathematical functions. In this project, I think I'll be using it for
# numerical operations like normalizing data.
import numpy as np

# pandas: A library for data manipulation and analysis. It provides data structures 
# like DataFrames, which are useful for organizing and analyzing structured data. While 
# not strictly necessary, it can be helpful if I need to inspect datasets or labels in 
# tabular format.
import pandas as pd

# os: A standard library for interacting with the file system. I'll be using it to navigate
# directories, list files, and perform file-related operations like loading datasets.
import os

# Image processing:
# PIL (Pillow): 
# A library for image processing. It allows the openning, manipulating, and 
# saving image files. In this project, it will be used to preprocess images, such as 
# resizing or cropping.
from PIL import Image

# torchvision.transforms: A module in PyTorch for preprocessing images. It provides 
# functions to resize, normalize, convert to tensors, and perform data augmentation 
# like rotation or flipping.
import torchvision.transforms as transforms

# torchvision.datasets: 
# Contains utilities for loading standard datasets like ImageNet, CIFAR-10, or custom datasets.
# torchvision.models: 
# Includes pre-trained deep learning models (e.g., ResNet, VGG). I can use these 
# models directly or fine-tune them for my flower classification task.
from torchvision import datasets, models

# nn: A module for building and training neural networks. It includes layers like 
# convolutional layers, activation functions, and loss functions.
# optim: A module for optimization algorithms (e.g., SGD, Adam). I'll use it to 
# adjust the model's parameters during training.
from torch import nn, optim

# Utilities and visualization
# matplotlib.pyplot: A library for creating visualizations. I’ll use it to plot images, 
# loss curves, or other visual representations of my data and model performance.
import matplotlib.pyplot as plt

# tqdm: A library for creating progress bars. It’s helpful for visualizing the progress
# of time-consuming tasks like training my model(s) or preprocessing images.
from tqdm import tqdm


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [24]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [25]:
# TODO: Define your transforms for the training, validation, and testing sets
# Defining transforms for preprocessing
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),  # Resize and crop to 224x224
        transforms.RandomHorizontalFlip(),  # Data augmentation
        transforms.ToTensor(),  # Convert to PyTorch tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),  # Resize to 256x256
        transforms.CenterCrop(224),  # Crop the center to 224x224
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([  # Similar to validation, without augmentation
        transforms.Resize(256),  # Resize to 256x256
        transforms.CenterCrop(224),  # Crop the center to 224x224
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# TODO: Load the datasets with ImageFolder
# Load datasets
image_datasets = {
    x: datasets.ImageFolder(
        root=f"{data_dir}/{x}",
        transform=data_transforms[x]
    )
    for x in ['train', 'valid', 'test']  # Include 'test' here
}

# TODO: Using the image datasets and the tranforms, define the dataloaders
# Define dataloaders 
dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=(x == 'train'))
    for x in ['train', 'valid', 'test']  
}


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [26]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [30]:
# TODO: Load pre-trained model     
from torchvision.models import vgg16, VGG16_Weights

# Load the pre-trained VGG16 model with updated syntax per 'depreciated' warning.
weights = VGG16_Weights.DEFAULT  # Use the most up-to-date weights
model = vgg16(weights=weights)

# Freeze feature extraction layers
for param in model.features.parameters():
    param.requires_grad = False

    


In [41]:
# Define a new feed-forward network
# 25088 is the size of the flattened output of VGG16 features.
# 102 is the number of flower categories in the dataset.
# Use ReLU activations and Dropout for regularization.
model.classifier = nn.Sequential(

# Input size matches VGG16 features output
nn.Linear(25088, 4096),  
nn.ReLU(),
nn.Dropout(0.5),
# Output size matches the number of flower categories
nn.Linear(4096, 102),  
nn.LogSoftmax(dim=1)
)

"""
In deep learning, "ReLU activation" refers to a commonly used activation function called Rectified Linear Unit, which outputs 
the maximum value between 0 and the input, while "Dropout" is a regularization technique that randomly "drops out" neurons
during training to prevent overfitting by encouraging the network to learn more robust features across different combinations 
of neurons; essentially, it forces the network to not rely too heavily on any single neuron. 
Key points about ReLU activation:
Function:
max(0, x) - This means any negative input is set to 0, and positive inputs remain unchanged. 
Benefits:
Computationally efficient, helps to avoid the "vanishing gradient" problem often seen with other activation functions like sigmoid. 
Key points about Dropout regularization:
How it works:
During training, a random subset of neurons in each layer are temporarily "dropped out" (set to zero) with a specified probability (dropout rate).
Effect:
By randomly removing neurons, the network is forced to learn more distributed representations and avoid overfitting to the training data. 
"""

'\nIn deep learning, "ReLU activation" refers to a commonly used activation function called Rectified Linear Unit, which outputs \nthe maximum value between 0 and the input, while "Dropout" is a regularization technique that randomly "drops out" neurons\nduring training to prevent overfitting by encouraging the network to learn more robust features across different combinations \nof neurons; essentially, it forces the network to not rely too heavily on any single neuron. \nKey points about ReLU activation:\nFunction:\nmax(0, x) - This means any negative input is set to 0, and positive inputs remain unchanged. \nBenefits:\nComputationally efficient, helps to avoid the "vanishing gradient" problem often seen with other activation functions like sigmoid. \nKey points about Dropout regularization:\nHow it works:\nDuring training, a random subset of neurons in each layer are temporarily "dropped out" (set to zero) with a specified probability (dropout rate).\nEffect:\nBy randomly removing n

In [39]:
# Loss function
# Use nn.CrossEntropyLoss (or nn.NLLLoss since I'm using LogSoftmax) and an optimizer like Adam or SGD:
criterion = nn.NLLLoss()

# Optimizer (only update classifier parameters)
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

'''
model.classifier.parameters() ensures that only the feed-forward classifier weights are updated during 
training, leaving the pre-trained feature extraction layers unchanged.
'''

'\nmodel.classifier.parameters() ensures that only the feed-forward classifier weights are updated during \ntraining, leaving the pre-trained feature extraction layers unchanged.\n'

In [40]:
for name, param in model.named_parameters():
    print(f"{name} requires_grad={param.requires_grad}")


features.0.weight requires_grad=False
features.0.bias requires_grad=False
features.2.weight requires_grad=False
features.2.bias requires_grad=False
features.5.weight requires_grad=False
features.5.bias requires_grad=False
features.7.weight requires_grad=False
features.7.bias requires_grad=False
features.10.weight requires_grad=False
features.10.bias requires_grad=False
features.12.weight requires_grad=False
features.12.bias requires_grad=False
features.14.weight requires_grad=False
features.14.bias requires_grad=False
features.17.weight requires_grad=False
features.17.bias requires_grad=False
features.19.weight requires_grad=False
features.19.bias requires_grad=False
features.21.weight requires_grad=False
features.21.bias requires_grad=False
features.24.weight requires_grad=False
features.24.bias requires_grad=False
features.26.weight requires_grad=False
features.26.bias requires_grad=False
features.28.weight requires_grad=False
features.28.bias requires_grad=False
classifier.0.weight 

In [ ]:
# TODO: Build and train your network
# Training and validation are done seperately for experimenting with some initial overfitting.
# Code also tracks loss and accuracy for both training and validation sets.
# 
# Move model to GPU (I'm on my local machine, but I have GPU hours through Udacity's Workspace).
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 10  # Total number of times the model will iterate over the entire training dataset.
# Number of epochs without validation: code skips validation and only performs training to potentially
# introduce slight overfitting and allow better learning.
train_only_epochs = 5  # 
steps = 0
running_loss = 0
print_every = 5

for epoch in range(epochs): # Iterates through each epoch.
    for inputs, labels in dataloaders['train']: # Iterates through batches of training data.
        steps += 1

        # Move data to device
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        logps = model(inputs) # Forward pass
        loss = criterion(logps, labels) # Calculate loss

        # Backward pass
        loss.backward()
        # Update model weights
        optimizer.step()

        running_loss += loss.item()

        # Only validate after train_only_epochs
        if epoch >= train_only_epochs and steps % print_every == 0:
            valid_loss = 0
            accuracy = 0
            model.eval()  # Switch to evaluation mode

            with torch.no_grad():
                for inputs, labels in dataloaders['valid']:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model(inputs)
                    batch_loss = criterion(logps, labels)
                    valid_loss += batch_loss.item()

                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Validation loss: {valid_loss/len(dataloaders['valid']):.3f}.. "
                  f"Validation accuracy: {accuracy/len(dataloaders['valid']):.3f}")
            running_loss = 0
            model.train()  # Switch back to training mode

    # Print loss during epochs with no validation
    if epoch < train_only_epochs:
        print(f"Epoch {epoch+1}/{epochs}.. Train loss: {running_loss/steps:.3f}")
        running_loss = 0


## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set
def test_model(model, dataloader, criterion):
    """
    Test the trained model on the test dataset.
    
    Args:
    - model: The trained PyTorch model.
    - dataloader: DataLoader for the test dataset.
    - criterion: Loss function used during training.
    
    Returns:
    - test_loss: Average loss on the test dataset.
    - test_accuracy: Accuracy on the test dataset.
    """
    model.eval()  # Switch model to evaluation mode
    test_loss = 0
    test_accuracy = 0

    with torch.no_grad():  # No gradient computation needed during testing
        for inputs, labels in dataloader:
            # Move data to the same device as the model
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            logps = model(inputs)
            loss = criterion(logps, labels)
            test_loss += loss.item()
            
            # Calculate accuracy
            ps = torch.exp(logps)  # Convert log probabilities to probabilities
            top_p, top_class = ps.topk(1, dim=1)  # Get the top predicted class
            equals = top_class == labels.view(*top_class.shape)  # Compare predictions
            test_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

    # Average loss and accuracy across the dataset
    test_loss = test_loss / len(dataloader)
    test_accuracy = test_accuracy / len(dataloader)

    print(f"Test Loss: {test_loss:.3f}")
    print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
    return test_loss, test_accuracy


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 
# Save the checkpoint after training the model and hyperparameters for reuse:
checkpoint = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'class_to_idx': image_datasets['train'].class_to_idx,
    'epochs': epochs,
    'classifier': model.classifier,
    'learning_rate': 0.001
}

torch.save(checkpoint, 'my_checkpoint.py')

'''
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'class_to_idx': image_datasets['train'].class_to_idx,
    'epochs': epochs,
    'classifier': model.classifier,
    'learning_rate': 0.001
}, 'checkpoint.pth')
'''

In [5]:
print(os.listdir())

['.github', '.gitignore', '.notebook', 'assets', 'cat_to_name.json', 'CODEOWNERS', 'flowers', 'Image Classifier Project.ipynb', 'LICENSE', 'predict.py', 'README.md', 'train.py']


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    
    # Load pre-trained model
    model = models.vgg16(pretrained=True)
    
    # Freeze feature extraction layers
    for param in model.features.parameters():
        param.requires_grad = False
    
    # Load the classifier from checkpoint
    model.classifier = checkpoint['classifier']
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # Load the optimizer state (if resuming training)
    optimizer = optim.Adam(model.classifier.parameters(), lr=checkpoint['learning_rate'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    # Load other info
    class_to_idx = checkpoint['class_to_idx']
    epochs = checkpoint['epochs']
    
    print("Checkpoint loaded successfully!")
    return model, optimizer, class_to_idx
'''
EXAMPLE USAGE OF CHECKPOINT LOADING: 
# Load the saved model and optimizer
model, optimizer, class_to_idx = load_checkpoint('checkpoint.pth')

# Move model to device (GPU if available)
model.to(device)
'''


# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image_path):
    """
    Scales, crops, and normalizes a PIL image for a PyTorch model.
    Returns a Numpy array.
    """
    
    # TODO: Process a PIL image for use in a PyTorch model
    # Load the image
    pil_image = Image.open(image_path)

    # Resize the image
    # Maintain aspect ratio, with the shortest side = 256 pixels
    pil_image = pil_image.resize((256, 256)) if pil_image.width < pil_image.height else pil_image.resize((256, 256))

    # Center crop the image to 224x224
    left = (pil_image.width - 224) / 2
    top = (pil_image.height - 224) / 2
    right = left + 224
    bottom = top + 224
    pil_image = pil_image.crop((left, top, right, bottom))

    # Convert the image to a Numpy array and scale the pixel values (0-1)
    np_image = np.array(pil_image) / 255.0

    # Normalize the image using the means and standard deviations for each channel
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    np_image = (np_image - mean) / std

    # Reorder the dimensions: from (height, width, color_channels) to (color_channels, height, width)
    np_image = np_image.transpose((2, 0, 1))

    return np_image


To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, top_k=5):
    """
    Predict the class (or classes) of an image using a trained deep learning model.

    Args:
        image_path (str): Path to the image.
        model (torch.nn.Module): Trained PyTorch model.
        top_k (int): Number of top predictions to return.

    Returns:
        probs (list): Probabilities of the top K classes.
        classes (list): Corresponding class labels for the top K probabilities.
    """
    
    # TODO: Implement the code to predict the class from an image file
    # Preprocess the image
    np_image = process_image(image_path)

    # Convert to a PyTorch tensor and add batch dimension
    image_tensor = torch.from_numpy(np_image).unsqueeze(0).float()

    # Move the model and input to the same device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    image_tensor = image_tensor.to(device)

    # Set the model to evaluation mode
    model.eval()

    # Forward pass
    with torch.no_grad():
        output = model(image_tensor)

    # Convert output to probabilities
    ps = torch.exp(output)

    # Get the top K probabilities and indices
    top_p, top_class = ps.topk(top_k, dim=1)

    # Convert to lists
    probs = top_p.cpu().numpy().flatten().tolist()
    classes = top_class.cpu().numpy().flatten().tolist()

    return probs, classes


In [ ]:
# TODO: Display an image along with the top 5 classes
# Assuming the model is already loaded and trained
image_path = "path_to_flower_image.jpg"

# Get the top 5 predictions
probs, classes = predict(image_path, model)

print("Probabilities:", probs)
print("Classes:", classes)



import matplotlib.pyplot as plt

def display_image_with_predictions(image_path, flower_names, probs):
    # Display the input image
    plt.figure(figsize=(6, 8))
    ax1 = plt.subplot(2, 1, 1)
    image = process_image(image_path)
    imshow(torch.from_numpy(image), ax=ax1)
    ax1.set_title(flower_names[0])  # Title with the top predicted flower name

    # Display a bar chart of probabilities
    ax2 = plt.subplot(2, 1, 2)
    y_pos = range(len(flower_names))
    ax2.barh(y_pos, probs, color="blue", align="center")
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(flower_names)
    ax2.invert_yaxis()
    ax2.set_xlabel("Probability")

    plt.tight_layout()
    plt.show()


## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
def sanity_check(image_path, model, cat_to_name, top_k=5):
    """
    Sanity check for the trained model by displaying the input image
    along with the top K predicted classes and their probabilities.

    Args:
        image_path (str): Path to the image.
        model (torch.nn.Module): Trained PyTorch model.
        cat_to_name (dict): Dictionary mapping class indices to flower names.
        top_k (int): Number of top predictions to display.
    """
    # Predict the top K classes
    probs, classes = predict(image_path, model, top_k=top_k)
    
    # Convert class indices to actual flower names
    flower_names = [cat_to_name[str(cls)] for cls in classes]
    
    # Process the image
    processed_image = process_image(image_path)
    
    # Plot the input image
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(4, 5))
    ax1 = imshow(torch.from_numpy(processed_image), ax=ax1)
    ax1.set_title(flower_names[0])  # Predicted flower name
    
    # Plot the probabilities as a horizontal bar chart
    y_pos = np.arange(len(flower_names))
    ax2.barh(y_pos, probs, align='center')
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(flower_names)
    ax2.invert_yaxis()  # Invert y-axis to show the highest probability at the top
    

In [ ]:
# Set x-axis ticks with increments of 0.1
# TODO: Display an image along with the top 5 classes 
def sanity_check(image_path, model, cat_to_name, top_k=5):
    
    """
    Sanity check for the trained model by displaying the input image
    along with the top K predicted classes and their probabilities.
    
    Args:
        image_path (str): Path to the image
        model (torch.nn.Module): Trained PyTorch model.
        cat_to_name (dict): Dictionary mapping class indices to flower names.
        top_k (int): Number of top predictions to display.
    """
    # Predict the top K classes
    probs, classes = predict(image_path, model, top_k=top_k)
    
    # Convert class indices to actual flower names
    flower_names = [cat_to_name[str(cls)] for cls in classes]
    
    # Process the image
    processed_image = process_image(image_path)
    
    # Plot the input image
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(4, 6))
    ax1 = imshow(torch.from_numpy(processed_image), ax=ax1)
    ax1.set_title(flower_names[0])  # Predicted flower name
    
    # Plot the probabilities as a horizontal bar chart
    y_pos = np.arange(len(flower_names))
    ax2.barh(y_pos, probs, align='center', color='skyblue')
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(flower_names)
    ax2.invert_yaxis()  # Invert y-axis to show the highest probability at the top
    
    # Set x-axis ticks with increments of 0.1 (matching the guide image)
    ax2.set_xticks(np.arange(0, 0.4, 0.1))
    ax2.set_xlim(0, 0.4)  # Set x-axis limit to match the maximum in your guide
    ax2.set_xlabel('Probability')
    
    plt.tight_layout()
    plt.show()

# Example usage
image_path = 'flowers/test/28/image_05230.jpg'
sanity_check(image_path, model, cat_to_name)


In [ ]:
# Locating files:

def find_file(image_name, search_dir):
    """
    Recursively search for a file in the directory and subdirectories.
    
    Args:
    - image_name (str): Name of the file to search for.
    - search_dir (str): Directory to search in.
    
    Returns:
    - str: The full path to the file if found, else None.
    """
    for root, dirs, files in os.walk(search_dir):
        if image_name in files:  # Use 'image_name' instead of 'file_name'
            return os.path.join(root, image_name)
    return None

# Example usage
image_name = 'image_05230.jpg'  # Replace with your image name
search_directory = 'flowers'  # Replace with the base directory to search

image_path = find_file(image_name, search_directory)

if image_path:
    print(f"Image found: {image_path}")
else:
    print(f"Image {image_name} not found in {search_directory}.")
    

# <center>Fari T. Lindo: AI Programming Project UML Flow Chart: Flower Classification </center>

```mermaid
graph TD
    A[Dataset Preparation] --> B[Load Dataset]
    B --> C[Preprocess Images]
    C --> D[Split into Train, Validate, and Test Sets]
    
    D --> E[Model Architecture]
    E --> F[Pre-trained VGG16 Model]
    F --> G[Freeze Layers]
    F --> H[Add Feedforward Classifier]
    
    E --> I[Training Workflow]
    I --> J[Train Model]
    I --> K[Validate Model]
    I --> L[Track Loss & Accuracy]
    
    D --> M[Testing Workflow]
    M --> N[Test Model on Unseen Data]
    N --> O[Accuracy]
    N --> P[Top Predictions]
    
    D --> Q[Inference Workflow]
    Q --> R[Predict Class from New Image]
    R --> S[Class Probabilities and Labels]



# <center> Fari T. LIndo: AI Programming Project - Flower Classification UML with Variables and Functions </center>

```mermaid
classDiagram
    class DatasetPreparation {
        - image_path: str
        - train_data: DataLoader
        - valid_data: DataLoader
        - test_data: DataLoader
        + load_dataset(path: str)
        + preprocess_images(image: PIL.Image)
        + split_data(train_ratio: float, valid_ratio: float)
    }

    class ModelArchitecture {
        - model: torch.nn.Module
        - classifier: torch.nn.Sequential
        + load_pretrained_model(weights: str)
        + freeze_layers()
        + add_feedforward_classifier(input_size: int, output_size: int)
    }

    class TrainingWorkflow {
        - criterion: torch.nn.Module
        - optimizer: torch.optim.Optimizer
        - epochs: int
        + train_model(data: DataLoader, model: torch.nn.Module)
        + validate_model(data: DataLoader, model: torch.nn.Module)
        + track_metrics(loss: float, accuracy: float)
    }

    class TestingWorkflow {
        + test_model(data: DataLoader, model: torch.nn.Module)
        + calculate_accuracy(predictions: Tensor, labels: Tensor)
    }

    class Inference {
        + process_image(image_path: str)
        + predict(image: PIL.Image, model: torch.nn.Module, top_k: int)
    }

    DatasetPreparation --> ModelArchitecture : Prepares input
    ModelArchitecture --> TrainingWorkflow : Provides model
    TrainingWorkflow --> TestingWorkflow : Trains and validates
    TestingWorkflow --> Inference : Final model
    Inference --> DatasetPreparation : Processes unseen data